# Multi-Component Vehicles and the Reparable Spares Notes

This notebook provides some explanation behind the logic behind the reparable spares model used to represent multi-component vehicles, and how to use the module functions.



Lets take the example of a fleet of cars owned by a delivery company. The company manages its maintenance repair and overhaul in house, and as such, is interested in how many spare parts it should procure for its fleet. The spares bought will affect the servicability of its fleet, and also the cost. 



# Node Model 

Each car can be approximated as a series of nodes, which are the components which make up the car.

So we can create a "Blueprint" model of the car out of all these nodes. 

The nodes can be modelled with a simple hierachical model:

Node Name, Part Type/Number, Child Nodes

eg

node- Car, Chasis, Children = [Engine, Chasis]

node - Engine, Engine V8, Children = [Oil Pump]

node - Chasis, Frame, Childern = [Left Front Wheel, Right Front Wheel, Left Rear WHeel, Right Rear Wheel]

node - Left Front Wheel, Wheel, Children = None

node - Right Front Wheel, Wheel, Children = None

node - Left Rear WHeel, Wheel, Children = None

node - Right Rear Wheel, Wheel, Children = None

node - Oil Pump, Oil Pump Model X, Children = None 

# Parts Populating Nodes

To have a functioning car, all the nodes in the blueprint need to be occupied by a distinct component which suits the specific node (i.e. Wheel in Left Rear Wheel Node) and is also serviceable.

So we need a separate class from the Nodes to house all the useful attribute information about the component.

One important attribute is whether a component is reparable or consumable, i.e. if it breaks or needs maintenance, do you spend the effort to fix it or just throw the part away and replace it with a new part.

Expensive parts tend to be reparable because of the sheer cost of procuring a new part. Consumables on the other hand tend to be cheap.


# Problem statements to solve

In general for a multi-component vehicle, a business owner would like to minimise his spending on spare parts for the components, while reaching certain serviceability levels which his customers are happy with. 

For consumable spares, since they are thrown away, the average rate of consumption can be calculated, and from there stock holding levels optimised against MOQ and re-stock times. A poisson model can be used to approximate the holdings needed. 

For reparable spares, since they are repaired, a more complex model is needed. The Turn Around Time for the component to be repaired also affects the serviceability of the fleet (which in turn affects revenue). Mistakes in procurement of reparable spares can also be costly to fix downstream (since they cost so much), and also have substantial procurement lead times (so they cannot be fixed overnight). 

So in general, there are two questions to answer for reparable spares:
1) Given a fixed budget, what is the ideal procurement number of each spare?
2) Given a varying scale of budgets, what serviceability of my fleet can I expect?

These are on contigent that the mission profile is done correctly. 



Lets go into the coding aspect of this.

First, import all the needed functions from the libraries

In [ ]:
#!pip install openpyxl
#!pip install matplotlib
# i dont think i used anything else

In [ ]:
import copy
import math

from logger import get_logger
from classes import Part_physical, Part_attributes, Car
from reader import load_part_attributes, load_blueprints
from mathstuff import gamma_approx, weibull_mean
from runners import do_one_run, do_first_allocation, get_new_service
from plotter import plot_partnumber_all, plot_partnumber_values, plot_budget_serv, plot_serv

logger = get_logger()

# Parts_data Class

The parts_data has attributes pertaining to a part number / component. Hereafter the two terms will be used interchangabily.

Objects in this class have the following key attributes:

part_object = Part_attributes(
    name=part_type,
    failure_hours=failure_hours,
    life_limit=life_limit,
    shape_factor=shape_factor,
    cost=cost,
    depot_overhaul=oh_limit,
    depot_tat=depot_tat,
    placeholder=placeholder
)

Some explanations for the less obvious attributes:

- depot_tat (depot turn around time),
- oh_limit (overhaul limit),
- Failure Hours (scale factor),
- and Shape factor. 

depot_tat is the turn around time which the part needs to spend at the depot to be fixed or serviced. 

Overhaul limits refer to scheduled timings for maintenance on the reparable part to restore it to full working condition. 

Shape and scale factor pertain to weibull distribution which describe the unscheduled timings for the repair, ie it describes when it breaks with respect to time. This is a more flexible way of describing failure modes to both technical and non technical people, the shape factor describes the type (eg infant mortality, random failure, or wearout)

These can be ready filled in with an excel programme or equivalent (save it as a csv)

Each row should consists of the following:
part_type, failure_hours, life_limit, oh_limit, shape_factor, cost, depot_tat, placeholder 

Lets take a look at the excel file called parts_data.


In [ ]:
part_objects = load_part_attributes()


part_objects
{'Vehicle_blueprint': PartBlueprint(Name: Vehicle, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False),

'Engine_blueprint': PartBlueprint(Name: Engine, Failure Hours: 1000, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: 10000, Shape Factor: 1, Depot TAT: 14, Cost: 1000, Placeholder: False),

'Chasis_blueprint': PartBlueprint(Name: Chasis, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False), 

'Piston_blueprint': PartBlueprint(Name: Piston, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False), 

'Wheel_blueprint': PartBlueprint(Name: Wheel, Failure Hours: 100, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: 80, Shape Factor: 3, Depot TAT: 7, Cost: 10, Placeholder: False)}

# Class blueprint 

After loading the parts, we can know load the blueprints for our car. 

Each row consists of the following: place, part_type, parent_place

This helps to build the nodes of the blueprint which we discussed earlier. 

Remeber when running any scripts that the part attributes should be loaded first. This tells the script what parts exist. If not the Blueprint will not work!


In [ ]:
car_blueprint = load_blueprints()

In [ ]:
car_blueprint
Place: Vehicle, Part Type: Vehicle
	Place: Engine, Part Type: Engine
		Place: Piston, Part Type: Piston
	Place: Chasis, Part Type: Chasis
		Place: Front Left Wheel, Part Type: Wheel
		Place: Front Right Wheel, Part Type: Wheel
		Place: Rear Left Wheel, Part Type: Wheel
		Place: Rear Right Wheel, Part Type: Wheel

There are also CSV versions in the reader module for quick importation

# Other Classes used

There are a couple of other classes in the module to help implement the running of a simulation.

Class Part - contains the instances of a specific part. serial number, part type, failure times, operating times etc. It must be created based on an existing part attribute!

Class Car - describes the instances of a specific multi-component created according to the specificed blueprint. contains all the parts used to make it up, as well as its overall serviceability

# applying this to a mission profile of a fleet of cars

The rough logic for the number of spare parts needed goes like this:

for each distinct part number
1) get the expected occurence of wear out on a daily basis
2) get the expected reparing turn around time
3) mulitply the two together. This is the number of spares you should need in order to sustain operations on a consistent basis.
4) assign budget based on the ratio of each part number.

If the ratio is below 1, then you will suffer from non-availability

This assumes that maintenance actions take less time than 1 day to remove and refit a new part. 

There is also unavoidance maintenance effort downtime will often render your availability of the fleet to be less than 100% on a daily basis. 

Since there is nothing you can do about this on a spares procurement basis, I normally exclude this from the calculations. So availability should be taken in this context as "before maintenance effort". 


So how do we go around using this via the module functions?

Lets set some variables:

In [ ]:
# set mission profile here
hours_per_day = 5
days = 1000
fleet_size = 5

Now we start to calculate the number of spares which we should need, following the logic template laid out earlier.

First, get the ratio of parts needed in one instance

In [ ]:
part_quantities = car_blueprint.get_part_quantities()

part_quantities
{'Vehicle': 1, 'Engine': 1, 'Piston': 1, 'Chasis': 1, 'Wheel': 4}

Next, get the MTBF for the parts.

In [ ]:
parts_needed = {}

for key, value in part_quantities.items():
    for parts in Part_attributes.master_list:
        if key == parts.name and parts.depot_tat is not None:
            mtbf = weibull_mean(parts.shape_factor, parts.failure_hours)
            zero_stoppages = value * fleet_size / mtbf * parts.depot_tat
            parts_needed[key] = [zero_stoppages, zero_stoppages * parts.cost]


In [ ]:
for k,v in parts_needed.items():
    print(k,v)
    
Vehicle [0.0, 0.0]
Engine [0.07295449845716712, 72.95449845716712]
Piston [0.0, 0.0]
Chasis [0.0, 0.0]
Wheel [1.6671773325635852, 16.671773325635854]

scale as per lowest common denominator, which will serve as the first estimate for allocating repairables budget.

Note that we only need the non zero items, which indicate that is it reparable.

In [ ]:
non_zero_parts = {key: value.copy() for key, value in parts_needed.items() if value[0] != 0}
lowest_part = min(non_zero_parts, key=lambda x: non_zero_parts[x][0])
lowest_value = non_zero_parts[lowest_part][0]

for key, value in non_zero_parts.items():
    # Append the result of value[1] / lowest_value rounded up
    rounded_value = math.ceil(value[0] / lowest_value)  # Example of using length of the string
    value.append(rounded_value)

In [ ]:
for k,v in non_zero_parts.items():
    print(k,v)
    
Engine [0.07295449845716712, 72.95449845716712, 1]
Wheel [1.6671773325635852, 16.671773325635854, 23]

lets do 1 run and see what the serviceability and parts look like

In [ ]:
budget = 1000

In [ ]:
temp_spares_allocated, budget = do_first_allocation(budget, part_quantities, non_zero_parts, fleet_size)

for k,v in temp_spares_allocated.items():
    print(k,v)
    
PartBlueprint(Name: Vehicle, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False) 5

PartBlueprint(Name: Engine, Failure Hours: 1000, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: 10000, Shape Factor: 1, Depot TAT: 14, Cost: 1000, Placeholder: False) 6

PartBlueprint(Name: Piston, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False) 5

PartBlueprint(Name: Chasis, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False) 5

PartBlueprint(Name: Wheel, Failure Hours: 100, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: 80, Shape Factor: 3, Depot TAT: 7, Cost: 10, Placeholder: False) 20


We can see that the code decided to spend all 1000 dollars on a spare engine. There is no budget left for the wheels. We will see later if this is the correct thing to do.

In [ ]:
spares_allocated = copy.deepcopy(temp_spares_allocated)
highest_servicability = 0

service_current, car_serviceable, car_breakage, serv_tracker, depot_tracker, warehouse_tracker, graveyard_tracker = do_one_run(
    spares_allocated, days, fleet_size, hours_per_day, car_blueprint)

In [ ]:
service_current
38.839999999999996

you can plot the data if you want from the plotter functions.


In [ ]:
#plot_serv(car_serviceable,car_breakage)

Now to optimise the allocation, if possible. we look at the budget left and try to change the combination of spares to see if it can be improved.

This is a bit bare bones because it relies on single monte carlo runs. It could be enhanced by doing it 30 times and then taking a certain percentile of servicebility. However it would take a considerable amount of time.

In [ ]:
for x2 in range(10):

    if service_current > highest_servicability:
        highest_servicability = service_current
        spares_allocated = copy.deepcopy(temp_spares_allocated)
        temp_spares_allocated, budget = get_new_service(days, fleet_size, part_quantities, spares_allocated,
                                                        serv_tracker, warehouse_tracker, budget)
        Car.reset_all_cars()
        Part_physical.reset_master_list()
        service_current, car_serviceable, car_breakage, serv_tracker, depot_tracker, warehouse_tracker, graveyard_tracker = do_one_run(
            spares_allocated, days, fleet_size, hours_per_day, car_blueprint)
    else:
        logger.info("no more improvements")
        break

In [ ]:
logger.warning(f"Current Serv {service_current}")
2025-07-23 22:21:43,180 - spares_tree - WARNING - Current Serv 38.339999999999996

Looks like it wasnt any good at improving.


for k,v in temp_spares_allocated.items():
    print(k,v)
    
PartBlueprint(Name: Vehicle, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False) 5

PartBlueprint(Name: Engine, Failure Hours: 1000, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: 10000, Shape Factor: 1, Depot TAT: 14, Cost: 1000, Placeholder: False) 6

PartBlueprint(Name: Piston, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False) 5

PartBlueprint(Name: Chasis, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False) 5

PartBlueprint(Name: Wheel, Failure Hours: 100, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: 80, Shape Factor: 3, Depot TAT: 7, Cost: 10, Placeholder: False) 20


things can get better if we increase the budget. lets do a simple run with the budget at 1000 and 1100 to see the difference.

In [ ]:
service_list.append(highest_servicability)
allocation_list.append(spares_allocated)

In [ ]:
budget_list = []
service_list = []
allocation_list = []

for budget in range(1000, 1200, 100):
    logger.warning(f"Running Budget {budget}")
    budget_list.append(budget)

    temp_spares_allocated, budget = do_first_allocation(budget, part_quantities, non_zero_parts, fleet_size)
    spares_allocated = copy.deepcopy(temp_spares_allocated)

    highest_servicability = 0

    service_current, car_serviceable, car_breakage, serv_tracker, depot_tracker, warehouse_tracker, graveyard_tracker = do_one_run(
        spares_allocated, days, fleet_size, hours_per_day, car_blueprint)

    logger.info(service_current)

    for x2 in range(10):

        if service_current > highest_servicability:
            highest_servicability = service_current
            spares_allocated = copy.deepcopy(temp_spares_allocated)
            temp_spares_allocated, budget = get_new_service(days, fleet_size, part_quantities, spares_allocated,
                                                            serv_tracker, warehouse_tracker, budget)
            Car.reset_all_cars()
            Part_physical.reset_master_list()
            service_current, car_serviceable, car_breakage, serv_tracker, depot_tracker, warehouse_tracker, graveyard_tracker = do_one_run(
                spares_allocated, days, fleet_size, hours_per_day, car_blueprint)
        else:
            logger.info("no more improvements")
            break
    print(highest_servicability)

    for key, value in spares_allocated.items():
        print(key,value)
    service_list.append(highest_servicability)
    allocation_list.append(spares_allocated)

    Car.reset_all_cars()
    Part_physical.reset_master_list()



92.24

PartBlueprint(Name: Vehicle, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False) 5

PartBlueprint(Name: Engine, Failure Hours: 1000, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: 10000, Shape Factor: 1, Depot TAT: 14, Cost: 1000, Placeholder: False) 6

PartBlueprint(Name: Piston, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False) 5

PartBlueprint(Name: Chasis, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False) 5

PartBlueprint(Name: Wheel, Failure Hours: 100, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: 80, Shape Factor: 3, Depot TAT: 7, Cost: 10, Placeholder: False) 20

97.0

PartBlueprint(Name: Vehicle, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False) 5

PartBlueprint(Name: Engine, Failure Hours: 1000, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: 10000, Shape Factor: 1, Depot TAT: 14, Cost: 1000, Placeholder: False) 6

PartBlueprint(Name: Piston, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False) 5

PartBlueprint(Name: Chasis, Failure Hours: inf, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: inf, Shape Factor: 1, Depot TAT: 0, Cost: 0, Placeholder: False) 5

PartBlueprint(Name: Wheel, Failure Hours: 100, Life Limit: inf, Depot Limit: inf, Depot Repair: inf, Depot Overhaul: 80, Shape Factor: 3, Depot TAT: 7, Cost: 10, Placeholder: False) 30


Horray, our algorithm decided to invest the extra 100 dollars in 10 tires, and it paid off in the terms on servicability.

In [ ]:
#plot_budget_serv(budget_list, service_list)
# graph the results if you want. 